In [ ]:
# 完善：
# - 获取个人主页account_id --> 保存数据库
# - 添加2fa验证 --> 保存数据库
# - 添加邮箱 --> 保存数据库
#   - 获取可用邮箱数据
#   - 读取邮件
#   - 提交验证
# - 完善资料
#   - 完善头像
#   - 完善主页



In [ ]:

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

def _connect_selenium(driverPath,debuggerAddress):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("debuggerAddress", debuggerAddress)

    chrome_service = Service(driverPath)
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

In [100]:
from tools.bit_api import fetchBrowserDetail, openBrowser
from tools.selenium_driver_helper import SeleniumDriverHelper

browser_id = '451407a7961e46ebaf66c4c6f7581140'

open_res = openBrowser(browser_id)
# self.logger.info(f'open driver[{browser_id}]-->{self.win_name}')

# open_res = fetchBrowserDetail(browser_id)
print(open_res['data'])
driverPath = open_res['data']['driver']
debuggerAddress = open_res['data']['http']
print(driverPath)
driver = _connect_selenium(driverPath,debuggerAddress)

helper = SeleniumDriverHelper(driver)

{'success': True, 'data': {'ws': 'ws://127.0.0.1:59375/devtools/browser/232b7e4a-f248-45f5-9143-47daa5c74b8c', 'http': '127.0.0.1:59375', 'coreVersion': '126', 'driver': 'C:\\Users\\lqj\\AppData\\Roaming\\BitBrowser\\chromedriver\\126\\chromedriver.exe', 'seq': 13, 'name': 'facebook-18621270071', 'remark': '18621270071,龙,雷,1996,10,13,wDRHzyuk0EPJy8M', 'groupId': '2c9bc04790aae7ca0190b5cb34182ea7', 'pid': 32936}}
{'ws': 'ws://127.0.0.1:59375/devtools/browser/232b7e4a-f248-45f5-9143-47daa5c74b8c', 'http': '127.0.0.1:59375', 'coreVersion': '126', 'driver': 'C:\\Users\\lqj\\AppData\\Roaming\\BitBrowser\\chromedriver\\126\\chromedriver.exe', 'seq': 13, 'name': 'facebook-18621270071', 'remark': '18621270071,龙,雷,1996,10,13,wDRHzyuk0EPJy8M', 'groupId': '2c9bc04790aae7ca0190b5cb34182ea7', 'pid': 32936}
C:\Users\lqj\AppData\Roaming\BitBrowser\chromedriver\126\chromedriver.exe


In [ ]:
def fetch_account_id(driver):
    driver.get('https://www.facebook.com/profile.php')
    if '?id=' in driver.current_url:
        return driver.current_url.split('=')[-1]
    else:
        return None
account_id = fetch_account_id(driver)
print(account_id)


In [ ]:

import time
from tools.sercury import get_2fa_code
# 添加2FA
def _open_2fa_bind(driver):
    # url = 'https://accountscenter.facebook.com/password_and_security'
    url = 'https://accountscenter.facebook.com/password_and_security/two_factor/'
    driver.get(url)

def _click_2fa_account():
    点击用户xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/div[4]/div/div/div[2]/div/div/div[1]'
    res = helper.find_or_fail(By.XPATH,点击用户xpath)
    res.click()
    time.sleep(2)

def _2fa_checkout_password(password):
    res = helper.find_or_fail(By.XPATH, '//input[@type="password"]')
    res.send_keys(password)

    密码确认按钮xpath = '/html/body/div[6]/div[1]/div/div[2]/div/div/div/div/div/div/div[4]/div[3]/div/div/div/div/div/div'
    time.sleep(2)
    res = helper.find_or_fail(By.XPATH,密码确认按钮xpath)
    res.click()
    time.sleep(2)

def _fetch_2fa_key() -> str:
    
    选择密保方式继续按钮xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div[3]/div/div/div/div/div/div/div/div'
    res = helper.find_or_fail(By.XPATH,选择密保方式继续按钮xpath)
    res.click()
    
    time.sleep(10)
    
    令牌密钥xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[3]/div/div[3]/div[2]/div[4]/div/div/div[4]/div/div[2]/div/div/div[1]/span'
    res = helper.find_or_fail(By.XPATH,令牌密钥xpath)
    otp_key = res.text
    
    令牌密钥下一步按钮xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[3]/div/div[4]/div[3]/div/div/div/div/div'
    res = helper.find_or_fail(By.XPATH,令牌密钥下一步按钮xpath)
    res.click()
    
    time.sleep(2)
    return otp_key

def _check_2fa_code(code):
    input_elements = helper.find_or_fail(By.XPATH, '//input[@type="text"]')
    input_elements.send_keys(code)
    time.sleep(3)
    令牌确认下一步按钮xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[4]/div/div[4]/div[3]/div/div/div/div/div/div/div/div'
    res = helper.find_or_fail(By.XPATH,令牌确认下一步按钮xpath)
    res.click()



In [101]:
_open_2fa_bind(driver)

In [102]:

_click_2fa_account()


//input[@type="password"] is not located. Please raise an issue with verbose=True


AttributeError: 'NoneType' object has no attribute 'send_keys'

In [103]:

password = 'wDRHzyuk0EPJy8M'
_2fa_checkout_password(password)

/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div[3]/div/div/div/div/div/div/div/div is not located. Please raise an issue with verbose=True


AttributeError: 'NoneType' object has no attribute 'click'

In [104]:

otp_key = _fetch_2fa_key()
print(otp_key)

6JGJ UQQJ JSRI NBA4 L47E PCMM 7KOR DP5I


In [105]:

otp_code = get_2fa_code(otp_code=otp_key)
print(otp_code)
_check_2fa_code(otp_code)

465815


In [ ]:
def _open_gmail_bind():
    url = 'https://accountscenter.facebook.com/personal_info/contact_points/'
    driver.get(url)

def _send_gmail_code(gmail):
    
    # 发送gmail验证码
    新增联络资料xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/div[4]/div/div/div[2]/div/div/div/div/div[2]/div[1]'
    res = helper.find_or_fail(By.XPATH,新增联络资料xpath)
    res.click()
    新增电子邮件xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/div[4]/div/div/div[2]/div[2]/div/div/div/div[2]/div[1]'
    res = helper.find_or_fail(By.XPATH,新增电子邮件xpath)
    res.click()
    input_elements = helper.find_or_fail(By.XPATH, '//input[@type="text"]')
    input_elements.send_keys(gmail)
    res = helper.find_or_fail(By.NAME,'noform')
    res.click()
    继续xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div[4]/div[3]/div/div/div/div/div/div/div/div'
    res = helper.find_or_fail(By.XPATH,继续xpath)
    res.click()


def _input_gmail_code(gmail_code):
    
    input_elements = helper.find_or_fail(By.XPATH, '//input[@type="text"]', return_elements=True)
    input_elements[1].send_keys(gmail_code)
    
    继续xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div/div/div/div[2]/div/div[4]/div[3]/div/div/div'
    res = helper.find_or_fail(By.XPATH,继续xpath)
    res.click()